# Data preparation

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import session_info

## Data load

In [2]:
B16F10 = pd.read_csv("../../data/tumor/B16F10.csv")
B16F10['Experiment'].unique()

array(['Hosoi_2014_Fig1a', 'Hosoi_2014_Fig5a', 'Hirano_2015_Fig1a',
       'Hirano_2015_Fig4a'], dtype=object)

In [3]:
B16F10 = pd.read_csv("../../data/tumor/B16F10.csv")
#low_tumor_set = ['Hosoi_2014_Fig1a', 'Hirano_2015_Fig1a', 'Hosoi_2014_Fig5a']
#B16F10 = B16F10[B16F10['Experiment'].isin(low_tumor_set)]

In [4]:
B16F10.head()

,Experiment,Condition,id,day,tumor_volume
0,Hosoi_2014_Fig1a,Untreated,1,-1,7.6
1,Hosoi_2014_Fig1a,Untreated,2,-1,17.4
2,Hosoi_2014_Fig1a,Untreated,3,-1,13.5
3,Hosoi_2014_Fig1a,Untreated,4,-1,12.2
4,Hosoi_2014_Fig1a,Untreated,5,-1,28.4


In [5]:
def tumor_volume(B16F10, condition):
    Treat = B16F10[(B16F10['Condition'] == condition)] 
    return Treat.groupby('day')['tumor_volume'].agg(['mean','std']).reset_index()

In [6]:
im_path = "../../data/immune/B16_immune_cells.csv"
B16IM = pd.read_csv(im_path)
B16IM["cells/mg"] = B16IM["cells/g"]/1e3
B16IM = B16IM.dropna(subset=['cells/mg']).reset_index(drop=True)
im_ave = pd.DataFrame()
im_ave = B16IM.groupby(['day','cell_type','Condition'])['cells/mg'].agg(['mean','std']).reset_index()

In [7]:
def im_data(condition, cell_type):
    return im_ave[(im_ave["cell_type"]==cell_type) & (im_ave["Condition"]==condition)]

In [8]:
g = 33.2
def immune_cell_count(condition, cell_type, g):
    """
    condition: Experiment condition(eg. "Untreated")
    cell_type: Cell type("CD45")
    g: density
    return: Dataframe[day, N_mean, N_std]
    """
    tv = tumor_volume(B16F10, condition).rename(
        columns={'mean':'V_mean', 'std':'V_std'}
    )
    im = im_data(condition, cell_type).rename(
        columns={'mean':'I_mean', 'std':'I_std'}
    )
    df = pd.merge(tv, im[['day','I_mean','I_std']], on='day')
    df['N_mean'] = g * df['V_mean'] * df['I_mean']
    df['N_var']  = (g**2) * (
        (df['I_mean']**2) * (df['V_std']**2) +
        (df['V_mean']**2) * (df['I_std']**2) +
        (df['V_std']**2) * (df['I_std']**2)
    )
    df['N_std'] = np.sqrt(df['N_var'])
    
    return df[['day','N_mean','N_std']]

In [9]:
unt_v_days = tumor_volume(B16F10, 'Untreated')['day']
unt_v_data = tumor_volume(B16F10, 'Untreated')['mean'].to_numpy()*1e6
unt_v_std = tumor_volume(B16F10, 'Untreated')['std'].to_numpy()*1e6

unt_mdsc = immune_cell_count("Untreated", "CD11bGr1", g)
unt_mdsc_days = np.array([-1, 1, 3, 5, 7])
unt_mdsc_data = np.concatenate(([(unt_v_data[0]/1e6)*g*im_data('Untreated','CD11bGr1')['mean'].to_numpy()[1]],
                                unt_mdsc['N_mean'].to_numpy()))
unt_mdsc_std = np.concatenate(([0],unt_mdsc['N_std'].to_numpy()))

In [10]:
# plt.figure(figsize=(4, 3))
# plt.errorbar(unt_v_days, unt_v_data, yerr=unt_v_std, fmt='o', capsize=3, markersize=3, color='black', label='Tumor')
# plt.errorbar(unt_mdsc['day'], unt_mdsc['N_mean'], yerr=unt_mdsc['N_std'], fmt='o', capsize=3, markersize=3, color='tab:green',label='MDSC')
# plt.xlabel('Days', fontsize=12)
# plt.ylabel('Cell Number', fontsize=12)
# #plt.title('Untreated')
# plt.yscale('log')
# plt.legend()
# plt.tight_layout()

In [11]:
# ===== CTL therapy =====
tst_mdsc = immune_cell_count("TST", "CD11bGr1", g)
tst_ctl = immune_cell_count("TST", "CTL", g)

v_days = np.array([-1, 1, 3, 5, 7, 9, 11, 13, 15])
v_data = tumor_volume(B16F10, 'TST')['mean'].to_numpy()*1e6
v_std = tumor_volume(B16F10, 'TST')['std'].to_numpy()*1e6

ctl_days = immune_cell_count("TST", "CTL", g)['day'].to_numpy()
ctl_data = immune_cell_count("TST", "CTL", g)['N_mean'].to_numpy()
ctl_std = immune_cell_count("TST", "CTL", g)['N_std'].to_numpy()

mdsc_days = np.array([-1, 1, 3, 5, 7])
mdsc_data = np.concatenate(([(v_data[0]/1e6)*g*im_data('TST','CD11bGr1')['mean'].to_numpy()[0]],
                            tst_mdsc['N_mean'].to_numpy()))
mdsc_std = np.concatenate(([0],tst_mdsc['N_std'].to_numpy()))

In [12]:
# plt.figure(figsize=(4, 3), dpi=300)
# plt.errorbar(v_days, v_data, yerr=v_std, fmt='o', capsize=3, markersize=3, color='black', label='Tumor')
# plt.errorbar(ctl_days, ctl_data, yerr=ctl_std, fmt='o', capsize=3, markersize=3, color='tab:blue',label='CTL')
# plt.errorbar(mdsc_days, mdsc_data, yerr=mdsc_std, fmt='o', capsize=3, markersize=3, color='tab:green',label='MDSC')
# plt.xlabel('Days', fontsize=12)
# plt.ylabel('Cell Number', fontsize=12)
# #plt.title('CTL therapy')
# plt.yscale('log')
# plt.legend()
# plt.tight_layout()